In [ ]:
import os
from pathlib import Path

In [ ]:
DOMAIN = 'www.data.gouv.fr'
PAGE_SIZE = 100
current_path = Path(os.getenv('WORKING_DIR')) if os.getenv('WORKING_DIR') else Path().absolute()
fn_path = '%s' % (current_path / 'moderation_fn.ipynb')
%run $fn_path
# we want everything that happened since this date
start_date = datetime(1970, 1, 1)
start_date

In [ ]:
harvesters = get_last_items('harvest/sources')
pending = [h for h in harvesters if h['validation']['state'] == 'pending']

In [ ]:
def show_harvesters(harvesters):
    for h in harvesters:
        txt = ""
        if not h['organization'] and not h['owner']:
            owner = '- orphan -'
            owner_url = '#'
        else:
            owner = h['organization']['name'] if h['organization'] else fullname(h['owner'])
            owner_url = h['organization']['page'] if h['organization'] else h['owner']['page']
        url = f'https://{DOMAIN}/fr/admin/harvester/{h["id"]}'
        
        txt += f":ear_of_rice: <{url}|{h['name']}>"
        txt += ' par '
        txt += f"<{owner_url}|{owner}>"
        txt += f"\n• Description: {h['description']}"
        txt += f"\n• Type: `{h['backend']}`"
        txt += f"\n• Created: `{h['created_at']}`"
        txt += f"\n• Url: <{h['url']}>"
        txt += f"\n• Filters: `{h['config'].get('filters')}`"
        
        yield txt

In [ ]:
txt = show_harvesters(pending)

In [ ]:
url = os.environ['NB_SLACK_WEBHOOK_URL_HARVEST']

def send_message(message):
    send_to_slack({'text': message})
                       
def send_to_slack(the_json):
    requests.post(url, json=the_json)

In [ ]:
if len(pending):
    send_to_slack({'text': f":tractor: *%s moissonneurs en attente* :arrow_down:" % len(pending)})
    for message in show_harvesters(pending):
        send_message(message)